In [40]:
import warnings
import pandas as pd
import vectorbt as vbt
import pandas_ta as pda
from tqdm import tqdm
from binance import Client

warnings.filterwarnings(action="ignore")

client = Client()

In [50]:
btc_klines = client.get_historical_klines(
    symbol="BTCUSDT",
    interval=Client.KLINE_INTERVAL_1DAY)

btc_klines_df = pd.DataFrame(btc_klines, columns=[
    "Open time", "Open", "High", "Low", 
    "Close", "Volume", "Close time", "Quote asset volume", 
    "Number of trades", "Taker buy base asset volume", 
    "Taker buy quote asset volume", "Ignore"
])

btc_klines_df['High'] = btc_klines_df['High'].astype(float)
btc_klines_df['Low'] = btc_klines_df['Low'].astype(float)
btc_klines_df['Close'] = btc_klines_df['Close'].astype(float)
btc_klines_df['Datetime'] = pd.to_datetime(btc_klines_df['Open time'], unit='ms')
btc_klines_df.set_index('Datetime', inplace=True)

In [51]:
btc_klines_df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
Datetime,,,,,,,,,,,,
2020-06-19,1592524800000,9386.32000000,9438.3,9215.79,9310.23,45330.98367300,1592611199999,423086965.19273827,503484,22055.96904600,205898368.34014912,0
2020-06-20,1592611200000,9310.23000000,9395.0,9170.95,9358.95,30329.06538400,1592697599999,282120360.55675261,382778,15125.08176900,140741525.62068328,0
2020-06-21,1592697600000,9358.95000000,9422.0,9281.54,9294.69,24316.92623400,1592783999999,227467940.61021120,321015,11779.79718200,110215504.55537862,0
2020-06-22,1592784000000,9294.69000000,9780.0,9277.09,9685.69,57895.46834300,1592870399999,550995799.06947046,588704,30738.20037500,292712778.14347612,0
2020-06-23,1592870400000,9685.69000000,9720.0,9577.03,9624.89,41031.02938000,1592956799999,395487616.46087799,474319,19112.60621600,184247416.84941235,0


In [52]:
periods = [x for x in range(2,80)]

result = []

for period in tqdm(periods):
    btc_hilo = pda.hilo(
        high=btc_klines_df['High'],
        low=btc_klines_df['Low'],
        close=btc_klines_df['Close'],
        high_length=period,
        low_length=period,
    )

    entries = btc_hilo[btc_hilo.keys()[1]].notnull()
    exits = btc_hilo[btc_hilo.keys()[2]].notnull()

    portfolio = vbt.Portfolio.from_signals(
        close=btc_klines_df['Close'],
        entries=entries,
        exits=exits,
        init_cash=500,
    )

    stats = portfolio.stats()
    stats["HILO Period"] = period
    result.append(stats)

100%|██████████| 78/78 [00:12<00:00,  6.33it/s]


In [53]:
result_df = pd.DataFrame(result)
result_df.sort_values("Total Return [%]", ascending=False).head(5)

,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,HILO Period
35,2020-06-19,2023-03-15,1000 days,500.0,2282.845182,356.569036,163.457831,100.0,0.0,56.287953,...,-8.386629,75 days 14:24:00,10 days 14:24:00,1.796195,116.281127,1.389307,1.315883,1.341407,2.188580,37
31,2020-06-19,2023-03-15,1000 days,500.0,2267.983771,353.596754,163.457831,100.0,0.0,51.967596,...,-8.313673,62 days 20:00:00,11 days 18:40:00,1.957787,115.307131,1.398748,1.417304,1.345033,2.193211,33
32,2020-06-19,2023-03-15,1000 days,500.0,2162.706825,332.541365,163.457831,100.0,0.0,51.967596,...,-8.313673,62 days 12:00:00,11 days 18:40:00,1.939436,108.407428,1.361945,1.359832,1.335057,2.132688,34
30,2020-06-19,2023-03-15,1000 days,500.0,2113.172035,322.634407,163.457831,100.0,0.0,50.678066,...,-8.046587,60 days 00:00:00,12 days 12:00:00,1.801922,98.588426,1.350529,1.366073,1.330568,2.103009,32
44,2020-06-19,2023-03-15,1000 days,500.0,2069.856061,313.971212,163.457831,100.0,0.0,50.828823,...,-9.556056,88 days 06:00:00,15 days 06:00:00,2.019774,127.902650,1.326948,1.336947,1.325127,2.076028,46
